## Solution to the second project

#### All the imports here

In [14]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from numpy import linspace, vstack, zeros, arange, power, abs
from mpl_toolkits.mplot3d import axes3d
from scipy.stats import norm
import random

#### Making matplotlib more tolerable

In [15]:
%matplotlib inline
# Font which got unicode math stuff.
mpl.rcParams['font.family'] = 'DejaVu Sans'
# Much more readable plots
plt.style.use('ggplot')

### Plotting functions below

In [16]:
#plotting functions here

### Task 2.1

In [17]:
#Code here

### Task 2.2

In [18]:
#Code here

### Task 2.3

In [19]:
#Code here

### Task 2.4

In [20]:
#Code here

### Task 2.5

In [21]:
#Code here